<a href="https://colab.research.google.com/github/Tomas201513/-Machine-learning-exercie/blob/main/Twiteer_dataset_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Normalization functions ⛏

In [1]:
def fix_text(text):
    text = text.replace('&amp;', '&')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    return text


ALLOW_NEW_LINES = False 
def clean_tweet(tweet, allow_new_lines = ALLOW_NEW_LINES):
        bad_start = ['http:', 'https:']
        for w in bad_start:
            tweet = re.sub(f" {w}\\S+", "", tweet)      # removes white space before url
            tweet = re.sub(f"{w}\\S+ ", "", tweet)      # in case a tweet starts with a url
            tweet = re.sub(f"\n{w}\\S+ ", "", tweet)    # in case the url is on a new line
            tweet = re.sub(f"\n{w}\\S+", "", tweet)     # in case the url is alone on a new line
            tweet = re.sub(f"{w}\\S+", "", tweet)       # any other case?
        tweet = re.sub(' +', ' ', tweet)                # replace multiple spaces with one space
        if not allow_new_lines:                         # TODO: predictions seem better without new lines
            tweet = ' '.join(tweet.split())
        return tweet.strip()

def boring_tweet(tweet):
      "Check if this is a boring tweet"
      boring_stuff = ['http', '@', '#']
      not_boring_words = len([None for w in tweet.split() if all(bs not in w.lower() for bs in boring_stuff)])
      return not_boring_words < 3

### Dataset maker 🔥

In [6]:
import json,re,urllib3,random

print("Enter acount\'s username without @ sign ❌\n")
handle=input().lower()

cool_tweets = []
handles_processed = []
raw_tweets = []
user_names = []
n_tweets_dl = []
n_retweets = []
n_short_tweets = []
n_tweets_kept = []
# clear_output(wait=True)
http = urllib3.PoolManager(retries=urllib3.Retry(3))
res = http.request("GET", f"http://us-central1-huggingtweets.cloudfunctions.net/get_tweets?handle={handle}&force=1")
res = json.loads(res.data.decode('utf-8'))

user_names.append(res['user_name'])
all_tweets = res['tweets']
raw_tweets.append(all_tweets)
curated_tweets = [fix_text(tweet) for tweet in all_tweets]

# create dataset
clean_tweets = [clean_tweet(tweet) for tweet in curated_tweets]
cool_tweets.append([tweet for tweet in clean_tweets if not boring_tweet(tweet)])

# save count
n_tweets_dl.append(str(res['n_tweets']))#number of total tweets
n_retweets.append(str(res['n_RT']))#number of re-tweets
n_short_tweets.append(str(len(all_tweets) - len(cool_tweets[-1]))) #number of short tweets
n_tweets_kept.append(str(len(cool_tweets[-1])))#

print(f"\n{n_tweets_dl[-1]} tweets downloaded, including {n_retweets[-1]} Re-tweets and {n_short_tweets[-1]} short tweets\nSaving {n_tweets_kept[-1]} tweets in 💥data_{handle}_train.txt💥 \n")

if len('<|endoftext|>'.join(cool_tweets[-1])) < 6000:
  # need about 4000 chars for one data sample (but depends on spaces, etc)
  raise ValueError(f"Error: this user does not have enough tweets to train a Neural Network\n{res['n_tweets']} tweets downloaded, including {res['n_RT']} RT's and {len(all_tweets) - len(cool_tweets)} boring tweets... only {len(cool_tweets)} tweets kept!")

seed_data = random.randint(0,2**32-1)
dataRandom = random.Random(seed_data)
total_text = '<|endoftext|>'
all_handle_tweets = []
epoch_len = max(len(''.join(cool_tweet)) for cool_tweet in cool_tweets)
EPOCHS = 4

for _ in range(EPOCHS):
    for cool_tweet in cool_tweets:
        dataRandom.shuffle(cool_tweet)
        current_tweet = cool_tweet
        current_len = len(''.join(current_tweet))
        while current_len < epoch_len:
            for t in cool_tweet:
                current_tweet.append(t)
                current_len += len(t)
                if current_len >= epoch_len: break
        dataRandom.shuffle(current_tweet)
        all_handle_tweets.extend(current_tweet)
total_text += '<|endoftext|>'.join(all_handle_tweets) + '<|endoftext|>'

with open(f"data_{handle}_train.txt", 'w') as f:
    f.write(total_text)

Enter acount's username without @ sign ❌

getfactet

289 tweets downloaded, including 8 Re-tweets and 64 short tweets
Saving 217 tweets in 💥data_getfactet_train.txt💥 

